In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

# Lecture 11 #

### Picking up where we left off last time
How could we take sex of the child into account when making predictions? 
Does that lead to smaller errors?

In [ ]:
# Read in data again and redefine variables and functions we used last time
families = Table.read_table('family_heights.csv')
parent_avgs = (families.column('father') + families.column('mother'))/2
def difference(x, y):
    return x - y
heights = Table().with_columns(
    'Parent Average', parent_avgs,
    'Child', families.column('child'),
    'Sex', families.column('sex')
)
heights

In [ ]:
def predict_smarter(h, s):
    nearby = heights...
    nearby_same_sex = nearby...
    return np.average(nearby_same_sex.column('Child'))

In [ ]:
predict_smarter(68, 'female')

In [ ]:
predict_smarter(68, 'male')

In [ ]:
smarter_predicted_heights = heights.apply(predict_smarter, 'Parent Average', 'Sex')
heights = heights.with_column('Smarter Prediction', smarter_predicted_heights)
heights

In [ ]:
smarter_pred_errs = heights.apply(difference, 'Child', 'Smarter Prediction')
heights = heights.with_column('Smarter Errors', smarter_pred_errs)
heights

In [ ]:
heights.hist('Smarter Errors', group='Sex')

## Lists

In [ ]:
[1, 5, 'hello', 5.0, make_array(1,2,3)]

### Rows from lists

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

In [ ]:
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

## Grouping by One Column ##

In [ ]:
cones = Table.read_table('cones.csv').drop('Color')
cones

In [ ]:
cones.group('Flavor')

In [ ]:
cones.group('Flavor', np.average)

In [ ]:
cones.group('Flavor', min)

## Grouping By One Column: Welcome Survey ##

In [ ]:
survey = Table.read_table('welcome_survey_fa23.csv')
survey.show(3)
def remove_plus(string):
    return string.strip(' or more')
survey = survey.with_columns('Number of textees',survey.apply(remove_plus,'Number of textees').astype(int))
survey = survey.where('Handedness',are.not_equal_to('nan'))

In [ ]:
survey.hist('Extroversion',bins = np.arange(0,11,1))

In [ ]:
survey.group('Extroversion')

In [ ]:
by_extra = survey.group('Extroversion', np.average)
by_extra

In [ ]:
by_extra.plot('Extroversion', 'Number of textees average')

In [ ]:
survey.group("Sleep position",np.average)

In [ ]:
survey.select("Sleep position", "Hours of sleep").group(0, np.average)

## Grouping by Two Columns ##

![Do right-handed people tend to sleep on their left side and left-handed people sleep on their right?](handed.png)

In [ ]:
survey.group(['Handedness', 'Sleep position'])

In [ ]:
survey.group(['Handedness', 'Sleep position'],
             np.average)

## Pivot Tables

In [ ]:
survey.group('Handedness')

In [ ]:
survey.pivot('Sleep position', 'Handedness')

In [ ]:
survey.pivot('Sleep position', 
             'Handedness', 
             values='Hours of sleep', 
             collect=np.average)

In [ ]:
# This cell will throw an error
# You cannot include just 1 of the optional arguments
survey.pivot('Sleep position', 
             'Handedness', 
             collect=np.average)

## Activity ##

In [ ]:
# From the CORGIS Dataset Project
# By Austin Cory Bart acbart@vt.edu
# Version 2.0.0, created 3/22/2016
# https://corgis-edu.github.io/corgis/csv/skyscrapers/

sky = Table.read_table('skyscrapers.csv')
sky = (sky.with_column('age', 2022 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

1. For each city, what’s the tallest building for each material?

In [ ]:
# 1. For each city, what’s the tallest building for each material?






2. For each city, what’s the height difference between the tallest steel building and the tallest concrete building?

In [ ]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?









3. Generate a table of the names of the oldest buildings for each material for each city:

In [ ]:
# 3. Generate a table of the names of the oldest buildings for each 
#    material for each city:

# Hint: You can use sort to find the name of the oldest building in the dataset
sky.sort('age', descending=True).column('name').item(0)


# Put your solution here








(No peeking! A solution appears below.)

In [ ]:
def first(s):
    "Return the first element in an array."
    return s.item(0)

(sky
 .sort('age', descending=True)
 .pivot('material', 'city', 'name', first)
)

## Joins ##

In [ ]:
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

In [ ]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

In [ ]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

In [ ]:
drinks.join('Cafe', drinks, 'Cafe')